In [1]:
%%capture
!pip install pypdf llama_index

In [2]:
import os
import subprocess
import yaml, os, openai
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [3]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['AD_OPENAI_API_KEY'] = credentials['AD_OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['AD_ENGINE'] = credentials['AD_ENGINE']

openai.api_key = credentials['AD_OPENAI_API_KEY']
openai.api_base = credentials['AD_OPENAI_API_BASE']
openai.api_type = credentials['AD_OPENAI_API_TYPE']
openai.api_version = credentials['AD_OPENAI_API_VERSION']
openai.engine = credentials['AD_ENGINE']

# Download Data

In [4]:
pdf_url = "https://browse.arxiv.org/pdf/1706.03762.pdf"

download_dir = "data/arxiv_pdf"
new_filename = "attention-is-all-you-need.pdf"

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

command = f"wget -O {download_dir}/attention-is-all-you-need.pdf {pdf_url}"

if len(os.listdir(download_dir)) == 0:
    try:
        subprocess.run(command, shell=True, check=True)
        print(f"PDF downloaded successfully to {download_dir}")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

# Load data and parse

In [11]:
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

chat_llm = OpenAI(
            # openai_api_key=os.environ["AD_OPENAI_API_KEY"],
            # engine = os.environ["AD_ENGINE"],
            # model='gpt-3.5-turbo',
            # temperature=0.0, 
            # max_tokens = 256
            )

In [12]:
documents = SimpleDirectoryReader(download_dir).load_data()
len(documents)

15

In [13]:
documents[0]

Document(id_='dfa7d12e-17ba-4468-bb95-cdc8efbc66c7', embedding=None, metadata={'page_label': '1', 'file_name': 'attention-is-all-you-need.pdf', 'file_path': 'data/arxiv_pdf/attention-is-all-you-need.pdf', 'creation_date': '2023-11-05', 'last_modified_date': '2023-08-05', 'last_accessed_date': '2023-11-05'}, excluded_embed_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='2ba68838d3d038f40da053d2e6e30a9e9b55e1f3e1a963d17cb1bbdf12ff76f4', text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGo

In [14]:
# Just use VectorStoreIndex of LlamaIndex itself
service_context = ServiceContext.from_defaults(
                                                embed_model=embedding_llm,
                                                llm=chat_llm
                                                )
llama_indexer = VectorStoreIndex.from_documents(
                                                documents, 
                                                service_context=service_context, 
                                                show_progress=True
                                                )

Parsing documents into nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

# Querying

In [15]:
query_engine = llama_indexer.as_query_engine(service_context=service_context) 

In [16]:
response = query_engine.query("What is the paper about ?")
response

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ce6e0819********************976c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}